In [5]:
import numpy as np 
import pandas as pd
import requests
from bs4 import BeautifulSoup

df_box_office_overall = pd.DataFrame(columns = ['within_year_rank',   # creating a template for a future dataset
                                                'title',
                                                'release_year',
                                                'worldwide_box_office',
                                                'domestic_box_office',
                                                'foreign_box_office']
)

for year in np.arange(1977,2024):
    url=f'https://www.boxofficemojo.com/year/world/{year}/'  # specific link for each year's page
    req=requests.get(url)  # creating a request
    content=req.text  # converting a reponse to text
    soup=BeautifulSoup(content)  # parsing
    all_tabular_records = soup.findAll('tr') # finding all tabular blocks on a page

    df_box_office_year = pd.DataFrame(columns = ['within_year_rank',   # creating a boofer df for a specific year
                                                'title',
                                                'release_year',
                                                'worldwide_box_office',
                                                'domestic_box_office',
                                                'foreign_box_office']
    )

    for i in range(len(all_tabular_records)):   # deriving required data from each record from a page's table, skipping a tables's header
        if i == 0:
            pass
        else:
            raw_data_row = all_tabular_records[i].findAll('td')   # finding tabular data
            data_raw = {}   # empty dict for further concatenating to the boofer df
            data_raw['within_year_rank'] = raw_data_row[0].text   # fetching a specific element of each 'bs4.element.ResultSet' and converting it to text
            data_raw['title'] = raw_data_row[1].text
            data_raw['release_year'] = year
            data_raw['worldwide_box_office'] = raw_data_row[2].text
            data_raw['domestic_box_office'] = raw_data_row[3].text
            data_raw['foreign_box_office'] = raw_data_row[5].text
            df_box_office_year = pd.concat([df_box_office_year, pd.DataFrame(data_raw, index = [0])]).reset_index(drop=True)   # concatenation of the line to the boofer df

    df_box_office_overall = pd.concat([df_box_office_overall, df_box_office_year])   # concatenation to the main df
    df_box_office_overall.index = np.arange(1, len(df_box_office_overall) + 1)   # resetting indices to follow the original rank numeration

df_box_office_overall

,within_year_rank,title,release_year,worldwide_box_office,domestic_box_office,foreign_box_office
1,1,Star Wars: Episode IV - A New Hope,1977,"$307,263,857","$307,263,857",-
2,2,Smokey and the Bandit,1977,"$126,737,428","$126,737,428",-
3,3,Close Encounters of the Third Kind,1977,"$116,395,460","$116,395,460",-
4,4,Saturday Night Fever,1977,"$94,213,184","$94,213,184",-
5,5,A Bridge Too Far,1977,"$50,750,000","$50,750,000",-
...,...,...,...,...,...,...
8437,197,God's Crooked Lines,2022,"$6,060,070",-,"$6,060,070"
8438,198,Notre-Dame on Fire,2022,"$6,059,323",-,"$6,059,323"
8439,199,Kingmaker,2022,"$6,049,522",-,"$6,049,522"
8440,200,Vikrant Rona,2022,"$6,020,595",-,"$6,020,595"


In [6]:
df_box_office_overall.info()   
# each record has an 'object' data type, for further processing it would be better to convert to integers where possible, as well as to avoid having '-' symbols for absent values

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8441 entries, 1 to 8441
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   within_year_rank      8441 non-null   object
 1   title                 8441 non-null   object
 2   release_year          8441 non-null   object
 3   worldwide_box_office  8441 non-null   object
 4   domestic_box_office   8441 non-null   object
 5   foreign_box_office    8441 non-null   object
dtypes: object(6)
memory usage: 461.6+ KB


In [7]:
for index, record in df_box_office_overall.iterrows():   # converting strings to numeric values for each line
    for column in ['worldwide_box_office', 'domestic_box_office', 'foreign_box_office']:
        if record[column] == '-':
            df_box_office_overall.at[index, column] = np.NaN
        else:
            df_box_office_overall.at[index, column] = int(df_box_office_overall.at[index, column].replace('$',"").replace(",",""))   # excluding surplus symbols from revenue values


for col_name in ['within_year_rank', 'release_year', 'worldwide_box_office', 'domestic_box_office', 'foreign_box_office']:   # optimizing numeric data types to minimum possible for specific columns
    df_box_office_overall[col_name] = pd.to_numeric(df_box_office_overall[col_name], errors='coerce', downcast='integer')

df_box_office_overall  # resulting df

,within_year_rank,title,release_year,worldwide_box_office,domestic_box_office,foreign_box_office
1,1,Star Wars: Episode IV - A New Hope,1977,307263857,307263857.0,NaN
2,2,Smokey and the Bandit,1977,126737428,126737428.0,NaN
3,3,Close Encounters of the Third Kind,1977,116395460,116395460.0,NaN
4,4,Saturday Night Fever,1977,94213184,94213184.0,NaN
5,5,A Bridge Too Far,1977,50750000,50750000.0,NaN
...,...,...,...,...,...,...
8437,197,God's Crooked Lines,2022,6060070,NaN,6060070.0
8438,198,Notre-Dame on Fire,2022,6059323,NaN,6059323.0
8439,199,Kingmaker,2022,6049522,NaN,6049522.0
8440,200,Vikrant Rona,2022,6020595,NaN,6020595.0


In [9]:
df_box_office_overall.info()   # cols 'domestic_box_office' and 'foreign_box_office' have 'float64' dtype due to NaN values in Serieses, others are Int16 or Int64, apart from title column

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8441 entries, 1 to 8441
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   within_year_rank      8441 non-null   int16  
 1   title                 8441 non-null   object 
 2   release_year          8441 non-null   int16  
 3   worldwide_box_office  8441 non-null   int64  
 4   domestic_box_office   7363 non-null   float64
 5   foreign_box_office    4814 non-null   float64
dtypes: float64(2), int16(2), int64(1), object(1)
memory usage: 620.7+ KB


In [11]:
df_box_office_overall.to_csv(r'/Users/artemohotnikov/Documents/__code/__pet_projects/Movies_project/Scripts/full_box_office_data.csv', index=False)   # exporting to csv